In [1]:
import os
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms


In [2]:
# Data augmentations and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images
        transforms.RandomHorizontalFlip(),  # Randomly flip images
        transforms.RandomRotation(10),  # Randomly rotate images
        transforms.ToTensor(),  # Convert images to tensors
        transforms.Normalize([0.485, 0.456, 0.406],  # Normalize with ImageNet stats
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images
        transforms.ToTensor(),  # Convert images to tensors
        transforms.Normalize([0.485, 0.456, 0.406],  # Normalize with ImageNet stats
                             [0.229, 0.224, 0.225])
    ]),
}

data_dir = r'path to\user_or_player'  # Replace with your dataset directory

# Create datasets
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}

# Create data loaders
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                   shuffle=True, num_workers=4)
    for x in ['train', 'val']
}

# Get dataset sizes and class names
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda:0


In [ ]:
# Load pre-trained model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Two output classes

model = model.to(device)


In [5]:
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification

optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


In [18]:
def train_model(model, criterion, optimizer, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Loop over epochs
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has training and validation phases
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimization in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Calculate epoch loss and accuracy
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it's the best so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()  # Newline for readability

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Validation Accuracy: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [19]:
num_epochs = 10  # Adjust based on your needs
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)


Epoch 1/10
----------
Train Loss: 0.3616 Acc: 0.8333
Val Loss: 11.7325 Acc: 0.4595

Epoch 2/10
----------
Train Loss: 0.0822 Acc: 0.9783
Val Loss: 49.2681 Acc: 0.4189

Epoch 3/10
----------
Train Loss: 0.0511 Acc: 0.9928
Val Loss: 26.5480 Acc: 0.4054

Epoch 4/10
----------
Train Loss: 0.0231 Acc: 0.9928
Val Loss: 4.1785 Acc: 0.5946

Epoch 5/10
----------
Train Loss: 0.0239 Acc: 0.9928
Val Loss: 0.2520 Acc: 0.9189

Epoch 6/10
----------
Train Loss: 0.0190 Acc: 0.9928
Val Loss: 0.0007 Acc: 1.0000

Epoch 7/10
----------
Train Loss: 0.0262 Acc: 0.9855
Val Loss: 0.0010 Acc: 1.0000

Epoch 8/10
----------
Train Loss: 0.0146 Acc: 1.0000
Val Loss: 0.0140 Acc: 1.0000

Epoch 9/10
----------
Train Loss: 0.0208 Acc: 0.9928
Val Loss: 0.0057 Acc: 1.0000

Epoch 10/10
----------
Train Loss: 0.0381 Acc: 0.9710
Val Loss: 0.0072 Acc: 1.0000

Training complete in 2m 31s
Best Validation Accuracy: 1.0000


In [20]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Collect predictions and labels
all_preds = []
all_labels = []

model.eval()  # Set model to evaluation mode

with torch.no_grad():
    for inputs, labels in dataloaders['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate classification report
print('Classification Report:')
print(classification_report(all_labels, all_preds, target_names=class_names))

# Generate confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print('Confusion Matrix:')
print(cm)


Classification Report:
              precision    recall  f1-score   support

  POV player       1.00      1.00      1.00        30
    POV user       1.00      1.00      1.00        44

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74

Confusion Matrix:
[[30  0]
 [ 0 44]]


In [21]:
torch.save(model.state_dict(), 'valorant_pov_classifier.pth')
print('Model saved successfully.')


Model saved successfully.


In [ ]:
import os
import shutil
import cv2
from PIL import Image
import torch
from torchvision import transforms

# Function to extract the first frame from a video
def extract_first_frame_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Failed to open video file: {video_path}")
        return None
    
    # Read the first frame
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        return Image.fromarray(frame)
    else:
        print(f"Failed to extract first frame from: {video_path}")
        return None

# Function to classify the first frame of the video
def predict_video_first_frame(video_path, model):
    model.eval()
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    frame = extract_first_frame_from_video(video_path)
    if frame is None:
        return None
    
    image = transform(frame).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        predicted_class = class_names[preds[0]]
    
    return predicted_class

# Function to move video to appropriate folder based on prediction
def classify_and_move_videos(input_folder, pov_user_folder, pov_player_folder, model):
    if not os.path.exists(pov_user_folder):
        os.makedirs(pov_user_folder)
    if not os.path.exists(pov_player_folder):
        os.makedirs(pov_player_folder)

    for video_file in os.listdir(input_folder):
        print(video_file)
        if video_file.endswith(('.mp4', '.avi', '.mkv', '.mov')):  # Add video formats you are using
            video_path = os.path.join(input_folder, video_file)
            
            # Classify the video based on the first frame
            prediction = predict_video_first_frame(video_path, model)
            print(prediction)
            if prediction is None:
                continue

            # Move the video to the correct folder based on the prediction
            if prediction == 'POV user':  # Assuming class names are 'user_pov' and 'player_pov'
                destination = os.path.join(pov_user_folder, video_file)
                shutil.copy2(video_path, destination)  # Use copy2 to preserve metadata
                print(f"Copied {video_file} to {pov_user_folder}")
            elif prediction == 'POV player':
                destination = os.path.join(pov_player_folder, video_file)
                shutil.copy2(video_path, destination)
                print(f"Copied {video_file} to {pov_player_folder}")


# Main coder
input_folder = r'path to\output\2024-09-26 21-56-16(Done)'  # Folder with your video clips
pov_user_folder = r'path to\clip/POV user'  # Folder to store classified User POV videos
pov_player_folder = r'path to\clip/POV player'  # Folder to store classified Player POV videos

# Assuming 'model' and 'class_names' are already defined as in the previous code
classify_and_move_videos(input_folder, pov_user_folder, pov_player_folder, model)
